In [2]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [1]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    time_format = '%Y-%h-%d:%H-%M-%S'
    current_time = datetime.now()
    timestamp = current_time.strfttime(time_format)
    with open('code_log.txt', 'a') as f:
        f.write(timestamp + ' : '+ str(message))
        
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    table_attribs = 
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

In [1]:
url = 'https://web.archive.org/web/20230908092635 /https://en.wikipedia.org/wiki/List_of_largest_banks'

In [3]:
webpage = requests.get(url).text

In [8]:
soup = BeautifulSoup(webpage,'html.parser')

In [11]:
table = soup.find('table')

In [30]:
table.find_all('tr')[3].find_all('td')[0].get_text().strip()

'3'

In [33]:
table.find_all('tr')[3].find_all('td')[1].get_text().strip()

'Industrial and Commercial Bank of China'

In [51]:
banks.keys()

dict_keys(['rank', 'bank_name', 'market'])

In [56]:
banks_dict = {'rank':[],'bank_name':[],'market':[]}

for col in table.find_all('tr'):
    if len(col.find_all('td')) == 0:
        continue
    for index,key in enumerate(banks.keys()):
        banks_dict[key].append(col.find_all('td')[index].get_text().strip())

banks_df = pd.DataFrame(banks_dict)

In [57]:
banks_df = pd.DataFrame()

,rank,bank_name,market
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [58]:
dict(['a','b','c'])

ValueError: dictionary update sequence element #0 has length 1; 2 is required